In [1]:
# Import Basis
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.preprocessing import StandardScaler as SS

from sklearn.model_selection import GridSearchCV as GCV

# Model
from sklearn.cluster import KMeans as KM, AgglomerativeClustering

# Model Evaluator
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [2]:
live = pd.read_csv('Live_cln_encd.csv')
live.sample(10)

,status_type,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys
765,1,217,1,0,215,2,0,0,0,0
985,1,302,9,0,302,0,0,0,0,0
5767,1,42,0,0,40,0,2,0,0,0
2254,1,14,23,0,14,0,0,0,0,0
3945,1,128,5,4,123,1,4,0,0,0
1603,1,326,9,0,326,0,0,0,0,0
4746,3,20,0,1,20,0,0,0,0,0
1000,1,136,5,1,136,0,0,0,0,0
4193,3,1,0,0,1,0,0,0,0,0
3734,1,5,0,0,5,0,0,0,0,0


In [3]:
kmeans1 = KM()

# Define the parameter grid to search
param_grid = {
    'n_clusters': range(1,11),
    'init' : ['k-means++', 'random'],
    'n_init': range(1,16),
    # 'max_iter': range(1,400),
    # 'tol': np.linspace(1e-5, 1e-3, 10),
    # 'verbose': range(0,10),
    # 'random_state': range(0,50),
    # 'copy_x': [True, False],
    # 'algorithm': ["lloyd", "elkan", "auto", "full"],
}

# Create GridSearchCV object
grid_model = GCV(
    estimator=kmeans1, 
    param_grid=param_grid, 
    cv=5, 
    return_train_score=True
)

# Fit the model
grid_model.fit(live)


# Get the best parameters
best_params = grid_model.best_params_
print(f"Best parameters found: {best_params}")

Best parameters found: {'init': 'k-means++', 'n_clusters': 10, 'n_init': 11}


In [12]:
# Convert cv_results_ to a DataFrame
results_df = pd.DataFrame(grid_model.cv_results_)
results_df.sample(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_init,param_n_clusters,param_n_init,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
225,0.009324,0.007613,0.000000,0.000000,random,6,1,"{'init': 'random', 'n_clusters': 6, 'n_init': 1}",-8.096915e+08,-6.648031e+07,...,-4.307975e+08,2.887190e+08,149,-7.661129e+08,-1.123709e+09,-9.919537e+08,-8.703031e+08,-1.014001e+09,-9.532161e+08,1.234063e+08
171,0.010395,0.008689,0.000000,0.000000,random,2,7,"{'init': 'random', 'n_clusters': 2, 'n_init': 7}",-2.480455e+09,-2.016815e+08,...,-1.453064e+09,1.001171e+09,268,-2.624002e+09,-5.354088e+09,-4.599785e+09,-4.227286e+09,-3.131593e+09,-3.987351e+09,9.891065e+08
247,0.061498,0.022766,0.002905,0.005810,random,7,8,"{'init': 'random', 'n_clusters': 7, 'n_init': 8}",-7.839269e+08,-5.527155e+07,...,-4.006703e+08,2.801120e+08,119,-6.577355e+08,-9.585676e+08,-8.708110e+08,-7.553439e+08,-8.003179e+08,-8.085552e+08,1.020053e+08
96,0.043274,0.006753,0.000000,0.000000,k-means++,7,7,"{'init': 'k-means++', 'n_clusters': 7, 'n_init...",-5.125055e+08,-6.832604e+07,...,-3.281400e+08,1.928336e+08,93,-6.372050e+08,-8.660600e+08,-7.785213e+08,-6.122250e+08,-5.591740e+08,-6.906371e+08,1.138269e+08
173,0.020141,0.006171,0.002974,0.005948,random,2,9,"{'init': 'random', 'n_clusters': 2, 'n_init': 9}",-2.480455e+09,-2.016815e+08,...,-1.337850e+09,1.072085e+09,244,-2.624002e+09,-5.354088e+09,-4.544491e+09,-4.227286e+09,-3.131588e+09,-3.976291e+09,9.824851e+08


In [13]:
kmeans = KM(
    n_clusters=10,
    init='k-means++',
    n_init=11
)
kmeans_labels = kmeans.fit_predict(live)

In [14]:
silhouette_avg = silhouette_score(live, kmeans_labels)
print("The average silhouette score is :", silhouette_avg)

The average silhouette score is : 0.6820234019228122


In [15]:
db_index = davies_bouldin_score(live, kmeans_labels)
print("The davies-bouldin index for is :", db_index)

The davies-bouldin index for is : 0.5835141434583972
